## **Setup**

In [ ]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
import pandas as pd

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

#hide
!pip install utils
from utils import *

     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 1.0MB 17.2MB/s 
     |████████████████████████████████| 358kB 32.0MB/s 
     |████████████████████████████████| 40kB 4.0MB/s 
     |████████████████████████████████| 92kB 10.7MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 51kB 3.3MB/s 
     |████████████████████████████████| 2.6MB 42.8MB/s 
ERROR: fastai 2.0.6 has requirement pandas>=1.1.0, but you'll have pandas 1.0.5 which is incompatible.


In [ ]:
from fastai.vision.all import *

In [ ]:
import os
import shutil
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
from PIL import Image

from sklearn.metrics import f1_score, accuracy_score

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

In [ ]:
# Set and test path to competition data files
competition_path = 'My Drive/AI For Good - AI Blitz 3/LNDST/Data/'
model_path='My Drive/AI For Good - AI Blitz 3/LNDST/Models/'
filename = 'train_images.zip'
assert(os.path.exists(f'{mount_path}/{competition_path}/{filename}'))
print('Drive mounted correctly and data accessible')

Drive mounted correctly and data accessible


# **Functions**



In [ ]:
def display_mask(imgnum):
  img=Image.open('/content/All_Data/Labels/train_gt/image_'+imgnum+'.png')
  img=np.asarray(img)*255
  display(Image.fromarray(np.uint8(img)))

# Unzip


In [ ]:
# All_Data is the home directory for the images, from which training and validation splits will be taken
if not os.path.exists('/content/All_Data/'):
  os.mkdir('/content/All_Data/')
  os.mkdir('/content/All_Data/Images/')
  os.mkdir('/content/All_Data/Labels/')

  with ZipFile(f'{mount_path}/{competition_path}/train_images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/All_Data/Images/')
  with ZipFile(f'{mount_path}/{competition_path}/train_gt.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/All_Data/Labels/')
    

In [ ]:
# imgnum=str(np.random.randint(1399))
# display_mask(imgnum)
# display(Image.open('/content/All_Data/Images/train_images/image_'+imgnum+'.jpg'))

In [ ]:
traindir='/content/All_Data/'

In [ ]:
def get_msk(itmpath):
  return os.path.join(traindir,f'Labels/train_gt/{itmpath.stem}.png')
  
# test
# get_msk(Path('/content/All_Data/Images/train_images/image_10.jpg'))

In [ ]:
def cvidxs(traindir,num_folds,val_pct,truly_random):
  # returns a dictionary[fold_num] : list of validation indices
  cvdict={key:[] for key in range(num_folds)}

  assert(val_pct<=(1/num_folds))

  if truly_random:
    rng = np.random.RandomState() # random seeding
  else:
    rng = np.random.RandomState(42) # deterministic seeding

  imgs=glob.glob(os.path.join(traindir,'Images/train_images/*.jpg'))
  imgs=np.arange(len(imgs))

  # shuffle the image array. NOTE all the randomness in the train-val split comes from this shuffle
  rng.shuffle(imgs)

  # size of the fold
  foldsz=np.floor(len(imgs)/num_folds).astype(int)
  # how many validation images
  numval=np.floor(val_pct*len(imgs)).astype(int)

  for cvfold in range(num_folds):
    valimgs=imgs[(cvfold*foldsz):(cvfold*foldsz+numval)]
    cvdict[cvfold]=valimgs
  
  return cvdict

Train the Model Zoo

In [ ]:
import gc

In [ ]:
models=[resnet18,xresnet.xresnet18,resnet34,xresnet.xresnet34,squeezenet1_0,squeezenet1_1]

for model in models:
  cvdict=cvidxs(traindir,3,0.15,True)
  for cvfold in range(3):
    dtblck=DataBlock(blocks=(ImageBlock, MaskBlock(['Background','Water'])),          
          get_items=get_image_files,
          splitter=IndexSplitter(cvdict[cvfold]),
          get_y=get_msk,
          batch_tfms=[*aug_transforms(max_rotate=30,min_zoom=0.5,max_zoom=1.5), Normalize.from_stats(*imagenet_stats)])

    dls=dtblck.dataloaders(source=traindir,bs=16)

    learn = unet_learner(dls, model, metrics=[Dice()])
    learn.to_fp16()
    learn.tta(n=10)

    learn.fine_tune(freeze_epochs=4,epochs=7,base_lr=1e-3)

    # learn.fit_one_cycle(1,1e-3)
    # assert(2==3)
    
    datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")
    learn.export(mount_path+model_path+datestr+'_'+str(model).split()[1]+'_cv'+str(cvfold)+'.pkl')

    del dtblck, dls, learn
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()



Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


epoch,train_loss,valid_loss,dice,time
0,0.502547,0.282619,0.874623,01:05
1,0.469528,0.326893,0.847571,01:05
2,0.412197,0.230755,0.904788,01:05
3,0.308016,0.220990,0.915973,01:05


epoch,train_loss,valid_loss,dice,time
0,0.230448,0.174221,0.930446,01:07
1,0.243752,0.175372,0.924110,01:07
2,0.177132,0.153887,0.939869,01:07
3,0.145961,0.138356,0.944199,01:07
4,0.120597,0.135072,0.946368,01:07
5,0.111407,0.126958,0.948493,01:07
6,0.106326,0.125000,0.949176,01:07


epoch,train_loss,valid_loss,dice,time
0,0.532634,0.224228,0.901462,01:05
1,0.467070,0.196309,0.922548,01:05
2,0.395093,0.208651,0.910247,01:05
3,0.322621,0.195941,0.929084,01:05


epoch,train_loss,valid_loss,dice,time
0,0.228088,0.160626,0.939340,01:07
1,0.227574,0.148408,0.944614,01:07
2,0.196786,0.146693,0.945099,01:07
3,0.143344,0.127483,0.950810,01:07
4,0.124047,0.128182,0.951213,01:07
5,0.111634,0.120812,0.952564,01:07
6,0.109109,0.118612,0.954397,01:07


...

epoch,train_loss,valid_loss,dice,time
0,0.487277,0.241836,0.899727,01:12
1,0.473818,0.220400,0.908333,01:12
2,0.467001,0.188544,0.921004,01:12
3,0.443894,0.174914,0.925483,01:12
4,0.446540,0.171640,0.930299,01:12
5,0.426525,0.154491,0.936087,01:12
6,0.402499,0.153763,0.935741,01:12


epoch,train_loss,valid_loss,dice,time
0,0.518541,0.239497,0.886770,01:09
1,0.515111,0.230297,0.893638,01:09
2,0.504178,0.225790,0.904746,01:09
3,0.509577,0.242810,0.899862,01:09


epoch,train_loss,valid_loss,dice,time
0,0.482898,0.192580,0.924495,01:12
1,0.467531,0.164168,0.934672,01:12
2,0.457698,0.152530,0.938270,01:12
3,0.439012,0.152833,0.938097,01:12
4,0.439358,0.140754,0.941611,01:12
5,0.402065,0.130832,0.946976,01:12
6,0.397534,0.130166,0.947270,01:12


Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-a815701f.pth


IndexError: ignored

In [ ]:
torch.cuda.device_count()